The first two notebooks cover the core functionality of `ptplot`, both to show you how fast you can get going but also to keep things from being overwhelming. This notebook shows off some of the more advanced or nonobvious ways you can use the package.

_(NOTE: If you are viewing this notebook on GitHub, none of the plots will render due to the [way GitHub processes the notebooks](https://github.com/plotly/plotly.py/issues/931). To see the notebook rendered properly, please [use nbviewer](https://nbviewer.jupyter.org/github/AndrewRook/ptplot/blob/main/notebooks/3-Tips_and_Tricks.ipynb).)_

Starting with imports and the same play as before:

In [1]:
import numpy as np
import pandas as pd

from ptplot import PTPlot
from ptplot.animation import Animation
from ptplot.hover import Hover
from ptplot.nfl import Aesthetics, Field
from ptplot.plot import Positions, Tracks

from bokeh.plotting import show
from bokeh.io import output_notebook

output_notebook()

# Load the data file
player_tracking_data = pd.read_csv(
    "2018_CLE_2018122305_1246.tsv",
    sep="\t", parse_dates=["time"]
)
player_tracking_data.loc[player_tracking_data["displayName"] == "ball", "teamAbbr"] = "ball"
player_tracking_data.loc[player_tracking_data["displayName"] == "ball", "jerseyNumber"] = ""
player_tracking_data.x = player_tracking_data.x - 10

# Just look between the snap and the tackle, to make the tracks clearer
snap_frame = player_tracking_data[player_tracking_data["event"] == "ball_snap"]["frame"].unique()[0]
tackle_frame = player_tracking_data[player_tracking_data["event"] == "tackle"]["frame"].unique()[0]
player_tracking_data = player_tracking_data[
    player_tracking_data["frame"].between(snap_frame, tackle_frame)
]
player_tracking_data.frame = player_tracking_data.frame - player_tracking_data.frame.min()

Loading BokehJS ...

## Fancy labels

Bokeh's default `"name": "data value"` hover labels are pretty good. But you can actually roll your own HTML entirely to make basically whatever you want. Here's an example that uses some additional player information to make little player cards, with headshots and some basic information:

In [2]:
roster_info = pd.read_csv("roster_2018.csv") # Downloaded from https://github.com/nflverse/nflfastR-roster

tracking_plus_roster_data = player_tracking_data.merge(
    roster_info, how="inner", left_on="gsisId", right_on="gsis_id",
    suffixes=("", "_roster")
)
tracking_plus_roster_data.loc[tracking_plus_roster_data.displayName == "ball", "full_name"] = "Ball"
tracking_plus_roster_data.loc[
    tracking_plus_roster_data.displayName == "ball", 
    ["headshot_url", "position", "jersey_number", "height", "weight"]
] = ""


tooltip = """
<div>
  <table>
    <tr>
      <td rowspan=3><img src="@headshot_url" height="42" alt="@headshot_url" width="42" 
                     style="float: left; margin: 0px 0px 0px 0px;" border="0">
      </img></td>
      <td><span style="font-size: 14px; font-weight: bold;">@full_name</span></td>
    </tr>
      <tr><td><span style="font-size: 11px; font-weight: bold;">#@jersey_number @position</span></td></tr>
      <tr><td><span style="font-size: 11px; font-weight: bold;">@height, @{weight} lbs</span></td></tr>
  </table>
</div>
"""


plot = (
    PTPlot(
        data=tracking_plus_roster_data, 
        pixel_height=400
    ) 
    + Field() 
    + Tracks("x", "y", "displayName", line_width=2)
    + Positions(
        "x", "y", number="jerseyNumber",
        name="positions"
    )
    + Aesthetics(team_ball_mapping="teamAbbr", home_away_mapping="homeTeamFlag == 1", ball_identifier="ball")
    + Hover(tooltip, "positions", ["headshot_url", "full_name", "jersey_number", "position", "height", "weight"])
    + Animation("frame", 10)
)
    
show(plot.draw())

## Calculated Mappings

You've already seen it in action when setting `"homeTeamFlag == 1"` in the `Aesthetics` instantiation, but the ability
to use a computed value instead of a variable name extends to _every_ mapping. For example, maybe you need to make the play go right-to-left:

In [3]:
plot = (
    PTPlot(
        data=player_tracking_data, 
        pixel_height=400
    ) 
    + Field() 
    + Tracks("100 - x", "y", "displayName", line_width=2)
    + Positions(
        "100 - x", "y", number="jerseyNumber",
        name="positions"
    )
    + Aesthetics(team_ball_mapping="teamAbbr", home_away_mapping="homeTeamFlag == 1", ball_identifier="ball")
    + Animation("frame", 10)
)
    
show(plot.draw())

## Weird Variable Names

Up to this point all of the variables used in the examples have been pretty "normal" — no spaces or other punctuation beyond underscores or dashes. But sometimes variables are not so nice, which can make life tough on `ptplot`'s underlying evaluation engine ([patsy](https://patsy.readthedocs.io/en/latest/overview.html)).

In [4]:
# Here we'll see what happens if the "x" column was actually written as "x position", and we wanted to use it 
# in an arithmetic expression:
player_tracking_data_weird_names = player_tracking_data.copy(deep=True)
player_tracking_data_weird_names["x position"] = player_tracking_data_weird_names["x"]

plot = (
    PTPlot(
        data=player_tracking_data_weird_names, 
        pixel_height=400
    ) 
    + Field() 
    + Tracks("100 - x position", "y", "displayName", line_width=2)
    + Positions(
        "100 - x position", "y", number="jerseyNumber",
        name="positions"
    )
    + Aesthetics(team_ball_mapping="teamAbbr", home_away_mapping="homeTeamFlag == 1", ball_identifier="ball")
    + Animation("frame", 10)
)

try:
    show(plot.draw())
except SyntaxError as e:
    print(e, e.text)

invalid syntax (<unknown>, line 1) I(100 - x position)



For those cases you'll want to [encase your variable names inside a `Q('')`](https://patsy.readthedocs.io/en/latest/builtins-reference.html?highlight=quoted#patsy.builtins.Q):

In [5]:
plot = (
    PTPlot(
        data=player_tracking_data_weird_names, 
        pixel_height=400
    ) 
    + Field() 
    + Tracks("100 - Q('x position')", "y", "displayName", line_width=2)
    + Positions(
        "100 - Q('x position')", "y", number="jerseyNumber",
        name="positions"
    )
    + Aesthetics(team_ball_mapping="teamAbbr", home_away_mapping="homeTeamFlag == 1", ball_identifier="ball")
    + Animation("frame", 10)
)
show(plot.draw())